In [1]:
%load_ext autoreload
%autoreload 2

import sys
HOME='/srv/home/christinedk/wp_internship/'
sys.path.append(HOME + 'collaboration/')
from config import TEMPLATES

In [2]:
import pandas as pd
import numpy as np
from time  import time
import matplotlib.pylab as plt
from pyspark.sql.functions import udf, col, explode, regexp_replace, first, mean, size, count, collect_list
import dateutil 
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import to_timestamp, min, max

from data_export import getTemplatesRegexRelaibility, getTemplatesRegex

In [3]:
TEMPLATES = ['weasel','peacock','autobiography','advert','fanpov']
outputHDFS = 'page_history'
!hadoop fs -mkdir $outputHDFS

mkdir: `page_history': File exists


In [4]:
## Read back revisions with template
revisions_with_template = spark.read.parquet('page_history/templates.parquet')

In [5]:
## Select subset of mediawiki history containing all page titles with revisions

pages_templates_subset = revisions_with_template.select('page_title').distinct()
pages_templates_subset.createOrReplaceTempView('pages_templates_subset')

users =  spark.sql('''
        SELECT DISTINCT w.event_user_id
        FROM wmf.mediawiki_history w
        WHERE w.snapshot ="2020-09" and w.wiki_db ="enwiki" AND  
        w.event_entity = 'revision' AND w.page_title IN (
        SELECT page_title FROM pages_templates_subset)  
        ''')
users.cache()
users = users.filter(users['event_user_id']!=7328338) # bad user

In [6]:
users.count()

405869

In [ ]:
#revisions_with_template.where(revisions_with_template['col']==template)

users = users.where(users['event_user_id'==])

In [48]:
users.createOrReplaceTempView('tmp_users')

In [49]:
print('extracting user histories')
t1 = time()
user_histories =  spark.sql('''
SELECT w.event_user_text, w.event_timestamp, w.page_title,w.page_id,w.page_namespace,
w.revision_id, w.revision_is_identity_reverted, revision_is_identity_revert,
w.revision_minor_edit, w.revision_text_bytes, 
w.revision_first_identity_reverting_revision_id,
w.event_user_id,w.event_user_registration_timestamp, 
SIZE(w.event_user_groups) as num_groups,
SIZE(w.event_user_blocks_historical) as num_blocks_historical, 
SIZE(w.event_user_blocks) as num_curr_blocks
FROM wmf.mediawiki_history w

WHERE w.snapshot ="2020-09" and w.wiki_db ="enwiki" AND  
w.event_entity = 'revision' 
AND w.event_user_id IN (SELECT event_user_id FROM tmp_users) 
AND SIZE(w.event_user_is_bot_by) = 0
ORDER BY w.event_user_id
''') 
#user_histories.cache()
user_histories.write.parquet('editors/user_histories.parquet',mode='overwrite')

extracting user histories
